<a href="https://colab.research.google.com/github/aquapathos/ColabHowto/blob/master/Real_timeGymMonitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-time Gym Monitor on google colaboratory by callback


In [1]:
import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64
import gym

env = gym.make('Pong-v0') 
out_img = env.reset()

#def run(img_str):
def run():
    action = env.action_space.sample()                  # 行動の決定
    out_img, reward, done, info = env.step(action) 

    #encode to string
    _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = encimg.tostring()
    img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
    return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)
# Image.fromarray(outimg)

In [2]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

def gmonitor():
  js = Javascript('''
    async function gCam() {
      const div = document.createElement('div');
      document.body.appendChild(div);

      const canvas = document.createElement('canvas');
      canvas.width  = 200;
      canvas.height = 210;
      const canvasCtx = canvas.getContext('2d')
      div.appendChild(canvas);
      
      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      //google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      var send_flg = false
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
             if (send_flg==false){
                send_flg = true
                const result = google.colab.kernel.invokeFunction('notebook.run', [], {});
                result.then(function(value) {
                    parse = JSON.parse(JSON.stringify(value))["data"]
                    parse = JSON.parse(JSON.stringify(parse))["application/json"]
                    parse = JSON.parse(JSON.stringify(parse))["img_str"]
                    var image = new Image()
                    image.src = parse;
                    image.onload = function(){canvasCtx.drawImage(image, 0, 0)}
                    send_flg = false
                })
            }
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);   
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
  display(js)
  data = eval_js('gCam()')

In [3]:
gmonitor()

<IPython.core.display.Javascript object>